In [1]:
import open3d as o3
import download
from open3d import JVisualizer
import os
import random
from random import randrange
from IPython.display import clear_output
import numpy as np
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
from datasets import PartDataset
from pointnet import PointNetAuto
import torch.nn.functional as F
import matplotlib.pyplot as plt
if torch.cuda.is_available():
    import torch.backends.cudnn as cudnn

In [2]:
# General parameters
NUM_POINTS = 10000
MODEL_PATH = './auto/auto_model_1.pth'
DATA_FOLDER = './shapenetcore_partanno_segmentation_benchmark_v0'

# download dataset and pre-trained model
# download.download_contents()

In [3]:
# Create dataset object
test_dataset = PartDataset(root = DATA_FOLDER, classification = False, train = False)
testdataloader = torch.utils.data.DataLoader(test_dataset, batch_size=1,
                                          shuffle=True, num_workers=0)

{'Mug': 11, 'Earphone': 5, 'Airplane': 0, 'Motorbike': 10, 'Pistol': 12, 'Cap': 2, 'Lamp': 8, 'Chair': 4, 'Car': 3, 'Laptop': 9, 'Rocket': 13, 'Table': 15, 'Knife': 7, 'Skateboard': 14, 'Bag': 1, 'Guitar': 6}


In [4]:
# Create the classification network from pre-trained model
classifier = PointNetAuto(k=3)

if torch.cuda.is_available():
    classifier.cuda()
    classifier.load_state_dict(torch.load(MODEL_PATH))
else:
    classifier.load_state_dict(torch.load(MODEL_PATH, map_location='cpu'))
classifier.eval()

PointNetAuto(
  (feat): PointNetfeat(
    (stn): STN3d(
      (conv1): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
      (conv2): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
      (conv3): Conv1d(128, 1024, kernel_size=(1,), stride=(1,))
      (mp1): MaxPool1d(kernel_size=2500, stride=2500, padding=0, dilation=1, ceil_mode=False)
      (fc1): Linear(in_features=1024, out_features=512, bias=True)
      (fc2): Linear(in_features=512, out_features=256, bias=True)
      (fc3): Linear(in_features=256, out_features=9, bias=True)
      (relu): ReLU()
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn3): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn5): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, tr

In [5]:
# Simple point cloud coloring mapping
def read_pointnet_colors(seg_labels):
    map_label_to_rgb = {
        1: [0, 255, 0],
        2: [0, 0, 255],
        3: [255, 0, 0],
        4: [255, 0, 255],  # purple
        5: [0, 255, 255],  # cyan
        6: [255, 255, 0],  # yellow
    }
    colors = np.array([map_label_to_rgb[label] for label in seg_labels])
    return colors

In [52]:
# Three.js based visualizer
visualizer = JVisualizer()

for test in testdataloader:
    # clean visualization
    visualizer.clear()
    clear_output()
    
    points, seg = test
    
    cloud = o3.PointCloud()
    cloud.points = o3.Vector3dVector(points[0])
    cloud.colors = o3.Vector3dVector(read_pointnet_colors(seg[0].numpy()))
    visualizer.add_geometry(cloud)

    points = Variable(points)
    points = points.transpose(2,1)
    if torch.cuda.is_available():
        points = points.cuda()
    
    pred, _ = classifier(points)
    t = pred[0].transpose(1, 0).cpu().detach()
    
    t[:, 2] += 1
    
    cloud = o3.PointCloud()
    cloud.points = o3.Vector3dVector(t)
    cloud.colors = o3.Vector3dVector(read_pointnet_colors(seg[0].numpy()))
    visualizer.add_geometry(cloud)
    visualizer.show()
    
    input()

JVisualizer with 2 geometries

KeyboardInterrupt: 

In [ ]:
torch.__version__